### Import libraries

In [1]:
#  import libraries
from bs4 import BeautifulSoup
import requests
import re #regex
import pandas as pd
from tqdm.notebook import tqdm
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
from lxml import etree
import lxml.html as lh

### Get url and ids list

In [2]:
# 2. find url and store it in avariable
url = "https://www.christies.com/lotfinder/print_sale.aspx?saleid=28619"

# 3. download html with a get request
response = requests.get(url)

#check response status code 
response.status_code

# create a beautiful soup
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
antiquities_list = [29064,29150,28840,29063,28980,
                    28619,28775,28620,28629,28618,28783,28785,28552,28917,28682,
                    28125,28597,28295,28580,28201,28097,28363,28178,28329,28200,
                    27821,27423,27522,27422,27942,27521,
                    26822,26920,26821,26919,
                    26158,27002,26172,26744,26157,26171,
                    25353,25123,25307,25122,
                    25050,24491,24743,24670,24618,24445,24628,
                    24269,24033,24150,24337,24554,23986,24219,24516,24326,
                    24268,23585,23820,17246,23545,23710,23760,
                    23097,23096,23264,23019,23214,23416,
                    22640,22639,22908,22631,22857,22937,22824,
                    22181,22180,22407,22490,22121,22384
                   ]
len(antiquities_list)

87

In [4]:
# get the estimated price (strip min and max by dividing by -)
soup.find_all("span",{"class":"lot-description"})[1].get_text(strip=True)

'£7,000 - £9,000'

### Get the data of one page 

In [5]:
all_data = [] 
for i in soup.find_all("tr"):
    all_data.append(i.text)
all_data = [x.strip().split("\n") for x in all_data]
all_data
#df = pd.DataFrame(all_data[1:])

[['55 Lots for Sale 18864'],
 ['Lot 4',
  'AN EGYPTIAN WHITE FAIENCE TILE WITH A REKHYT BIRD',
  'NEW KINGDOM, 19TH-20TH DYNASTY, CIRCA 1292-1069 B.C.',
  '3 5/8 X 4 in. (9.2 x 10.2 cm.)',
  '',
  'estimate £7,000 - £9,000($9,324 - $11,988) ',
  'price realized £8,750  ($11,778)'],
 ['Lot 5',
  'AN EGYPTIAN POLYCHROME WOOD FIGURE OF AN OFFICIAL',
  'MIDDLE KINGDOM, EARLY 12TH DYNASTY, CIRCA 1981-1802 B.C.',
  '12 3/8 in. (31.5 cm.) high',
  '',
  'estimate £30,000 - £50,000($39,960 - $66,600) ',
  'price realized £35,000  ($47,110)'],
 ['Lot 6',
  'AN EGYPTIAN PAINTED CARTONNAGE MUMMY MASK',
  'NEW KINGDOM, 19TH DYNASTY, CIRCA 1292-1185 B.C.',
  '8 ½ in. (21.5 cm.) high',
  '',
  'estimate £15,000 - £20,000($19,980 - $26,640) ',
  'price realized £18,750  ($25,238)'],
 ['Lot 7',
  'AN EGYPTIAN TURQUOISE FAIENCE FLASK',
  'NEW KINGDOM, 18TH DYNASTY, CIRCA 1550-1292 B.C.',
  '5 in. (12.8 cm.) high',
  '',
  'estimate £15,000 - £25,000($19,980 - $33,300) ',
  'price realized £4,375  ($5,8

In [7]:
# this gets the data of one page lots 
url = "https://www.christies.com/lotfinder/print_sale.aspx?saleid=28619"
response = requests.get(url)
response.status_code
soup = BeautifulSoup(response.content, 'html.parser')

lot_data = [] 
for i in soup.find_all("tr"):
    lot_data.append(i.text)
lot_data = [x.strip().split("\n") for x in lot_data]
df_lot = pd.DataFrame(lot_data[1:])#, columns=['lot','title','descr','measures','empty','estimated','realizes'])



head = [[
        #auction title/cat: 
        re.sub("[\n\r/]", "", soup.find_all("h2",{"class":"browse-sale-title"})[0].get_text(strip=True)),
        #auction id: 
        re.sub("[\n\r/]", "", soup.find_all("span",{"class":"sale-number"})[0].get_text(strip=True)),
        #date: 
        re.sub("[\n\r/]", "", soup.find_all("span",{"class":"sale-date"})[0].get_text(strip=True)),
        #location: 
        re.sub("[\n\r/]", "", soup.find_all("span",{"class":"sale-date"})[0].get_text(strip=True)),
        #total sale: 
        re.sub("[\n\r/]", "", soup.find_all("span",{"id":"spanTotal"})[0].get_text(strip=True)),
        #price realized: 
        re.sub("[\n\r/]", "", soup.find_all("div",{"id":"lots_for_sale"},"strong")[0].get_text(strip=True))
        ]]
df_head = pd.DataFrame(head*(len(df_lot)), columns = ['category','id','date','location','total_sale','nooflots'])

result = pd.concat([df_head, df_lot], axis=1, join="inner")

In [9]:
result

,category,id,date,location,total_sale,nooflots,0,1,2,3,4,5,6,7,8
0,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864,Lot 4,AN EGYPTIAN WHITE FAIENCE TILE WITH A REKHYT BIRD,"NEW KINGDOM, 19TH-20TH DYNASTY, CIRCA 1292-106...",3 5/8 X 4 in. (9.2 x 10.2 cm.),,"estimate £7,000 - £9,000($9,324 - $11,988)","price realized £8,750 ($11,778)",None,None
1,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864,Lot 5,AN EGYPTIAN POLYCHROME WOOD FIGURE OF AN OFFICIAL,"MIDDLE KINGDOM, EARLY 12TH DYNASTY, CIRCA 1981...",12 3/8 in. (31.5 cm.) high,,"estimate £30,000 - £50,000($39,960 - $66,600)","price realized £35,000 ($47,110)",None,None
2,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864,Lot 6,AN EGYPTIAN PAINTED CARTONNAGE MUMMY MASK,"NEW KINGDOM, 19TH DYNASTY, CIRCA 1292-1185 B.C.",8 ½ in. (21.5 cm.) high,,"estimate £15,000 - £20,000($19,980 - $26,640)","price realized £18,750 ($25,238)",None,None
3,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864,Lot 7,AN EGYPTIAN TURQUOISE FAIENCE FLASK,"NEW KINGDOM, 18TH DYNASTY, CIRCA 1550-1292 B.C.",5 in. (12.8 cm.) high,,"estimate £15,000 - £25,000($19,980 - $33,300)","price realized £4,375 ($5,889)",None,None
4,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864,Lot 9,AN EGYPTIAN NUMMULITIC LIMESTONE FIGURE OF AN ...,"NEW KINGDOM, 18TH DYNASTY, REIGN OF AKHENATEN,...",13 ½ in. (34.3 cm.) high,,"estimate ($532,800 - $799,200) This lot was wi...",None,None,None
5,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864,Lot 11,AN EGYPTIAN BRONZE CAT,"PTOLEMAIC PERIOD, CIRCA 332-30 B.C.",6 5/16 in. (16 cm.) high,,"estimate £15,000 - £25,000($19,980 - $33,300)","price realized £27,500 ($37,015)",None,None
6,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864,Lot 13,AN EGYPTIAN BRONZE OSIRIS-PTAH,"LATE PERIOD, 26TH DYNASTY, REIGN OF PSAMTIK I,...",8 ¼ in. (21 cm.) high,,"estimate £80,000 - £120,000($106,560 - $159,840)","price realized £387,500 ($521,575)",None,None
7,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864,Lot 15,A CYPRIOT LIMESTONE HEAD OF A MALE VOTARY,"ARCHAIC PERIOD, CIRCA LATE 6TH CENTURY B.C.",7 5/16 in. (18.6 cm.) high,,"estimate £40,000 - £60,000($53,280 - $79,920)","price realized £50,000 ($67,300)",None,None
8,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864,Lot 16,A CYPRIOT POTTERY ANTHROPOMORPHIC JUG,"CYPRO-ARCHAIC I-II, CIRCA 600-480 B.C.",10 ¾ in. (27.3 cm.) high,,"estimate £6,000 - £8,000($7,992 - $10,656)","price realized £21,250 ($28,603)",None,None
9,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864,Lot 17,A CYPRIOT BICHROME POTTERY JUG,"ARCHAIC PERIOD, CIRCA LATE 6TH-EARLY 5TH CENTU...",14 1/8 in. (36.1 cm.) high,,"estimate £6,000 - £8,000($7,992 - $10,656)","price realized £12,500 ($16,825)",None,None


In [10]:
# print the length of i in lot_data
for i in lot_data:
    print(len(i))

1
7
7
7
7
6
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
6
7
7
7
7
7
7
6
7
7
7
7
7
7
7
7
7
7
9
7
7
7
7
7
7
7


In [11]:
for i in lot_data:                
    while len(i) == 7:
        print(len(i))
        break 

7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7


-------

## Separately attempt 

In [2]:
# this gets the data of the single lots 
url = "https://www.christies.com/lotfinder/print_sale.aspx?saleid=28619"
response = requests.get(url)
response.status_code
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
all_data = [] 
for i in soup.find_all("tr"):
    all_data.append(i.text)
all_data = [x.strip().split("\n") for x in all_data]
all_data
#df = pd.DataFrame(all_data[1:])

[['55 Lots for Sale 18864'],
 ['Lot 4',
  'AN EGYPTIAN WHITE FAIENCE TILE WITH A REKHYT BIRD',
  'NEW KINGDOM, 19TH-20TH DYNASTY, CIRCA 1292-1069 B.C.',
  '3 5/8 X 4 in. (9.2 x 10.2 cm.)',
  '',
  'estimate £7,000 - £9,000($9,324 - $11,988) ',
  'price realized £8,750  ($11,778)'],
 ['Lot 5',
  'AN EGYPTIAN POLYCHROME WOOD FIGURE OF AN OFFICIAL',
  'MIDDLE KINGDOM, EARLY 12TH DYNASTY, CIRCA 1981-1802 B.C.',
  '12 3/8 in. (31.5 cm.) high',
  '',
  'estimate £30,000 - £50,000($39,960 - $66,600) ',
  'price realized £35,000  ($47,110)'],
 ['Lot 6',
  'AN EGYPTIAN PAINTED CARTONNAGE MUMMY MASK',
  'NEW KINGDOM, 19TH DYNASTY, CIRCA 1292-1185 B.C.',
  '8 ½ in. (21.5 cm.) high',
  '',
  'estimate £15,000 - £20,000($19,980 - $26,640) ',
  'price realized £18,750  ($25,238)'],
 ['Lot 7',
  'AN EGYPTIAN TURQUOISE FAIENCE FLASK',
  'NEW KINGDOM, 18TH DYNASTY, CIRCA 1550-1292 B.C.',
  '5 in. (12.8 cm.) high',
  '',
  'estimate £15,000 - £25,000($19,980 - $33,300) ',
  'price realized £4,375  ($5,8

In [4]:
len(all_data)

56

In [5]:
clean_data = []
for i in all_data:
    lent=len(i)
    if lent==7:
        clean_data.append(i)

In [6]:
clean_data

[['Lot 4',
  'AN EGYPTIAN WHITE FAIENCE TILE WITH A REKHYT BIRD',
  'NEW KINGDOM, 19TH-20TH DYNASTY, CIRCA 1292-1069 B.C.',
  '3 5/8 X 4 in. (9.2 x 10.2 cm.)',
  '',
  'estimate £7,000 - £9,000($9,324 - $11,988) ',
  'price realized £8,750  ($11,778)'],
 ['Lot 5',
  'AN EGYPTIAN POLYCHROME WOOD FIGURE OF AN OFFICIAL',
  'MIDDLE KINGDOM, EARLY 12TH DYNASTY, CIRCA 1981-1802 B.C.',
  '12 3/8 in. (31.5 cm.) high',
  '',
  'estimate £30,000 - £50,000($39,960 - $66,600) ',
  'price realized £35,000  ($47,110)'],
 ['Lot 6',
  'AN EGYPTIAN PAINTED CARTONNAGE MUMMY MASK',
  'NEW KINGDOM, 19TH DYNASTY, CIRCA 1292-1185 B.C.',
  '8 ½ in. (21.5 cm.) high',
  '',
  'estimate £15,000 - £20,000($19,980 - $26,640) ',
  'price realized £18,750  ($25,238)'],
 ['Lot 7',
  'AN EGYPTIAN TURQUOISE FAIENCE FLASK',
  'NEW KINGDOM, 18TH DYNASTY, CIRCA 1550-1292 B.C.',
  '5 in. (12.8 cm.) high',
  '',
  'estimate £15,000 - £25,000($19,980 - $33,300) ',
  'price realized £4,375  ($5,889)'],
 ['Lot 11',
  'AN EGYP

In [7]:
len(clean_data)

51

In [19]:
lot_data = pd.DataFrame.from_records(clean_data, columns = ['lot','title','descr','measures','empty','estimated','realized'])
lot_data

,lot,title,descr,measures,empty,estimated,realized
0,Lot 4,AN EGYPTIAN WHITE FAIENCE TILE WITH A REKHYT BIRD,"NEW KINGDOM, 19TH-20TH DYNASTY, CIRCA 1292-106...",3 5/8 X 4 in. (9.2 x 10.2 cm.),,"estimate £7,000 - £9,000($9,324 - $11,988)","price realized £8,750 ($11,778)"
1,Lot 5,AN EGYPTIAN POLYCHROME WOOD FIGURE OF AN OFFICIAL,"MIDDLE KINGDOM, EARLY 12TH DYNASTY, CIRCA 1981...",12 3/8 in. (31.5 cm.) high,,"estimate £30,000 - £50,000($39,960 - $66,600)","price realized £35,000 ($47,110)"
2,Lot 6,AN EGYPTIAN PAINTED CARTONNAGE MUMMY MASK,"NEW KINGDOM, 19TH DYNASTY, CIRCA 1292-1185 B.C.",8 ½ in. (21.5 cm.) high,,"estimate £15,000 - £20,000($19,980 - $26,640)","price realized £18,750 ($25,238)"
3,Lot 7,AN EGYPTIAN TURQUOISE FAIENCE FLASK,"NEW KINGDOM, 18TH DYNASTY, CIRCA 1550-1292 B.C.",5 in. (12.8 cm.) high,,"estimate £15,000 - £25,000($19,980 - $33,300)","price realized £4,375 ($5,889)"
4,Lot 11,AN EGYPTIAN BRONZE CAT,"PTOLEMAIC PERIOD, CIRCA 332-30 B.C.",6 5/16 in. (16 cm.) high,,"estimate £15,000 - £25,000($19,980 - $33,300)","price realized £27,500 ($37,015)"
5,Lot 13,AN EGYPTIAN BRONZE OSIRIS-PTAH,"LATE PERIOD, 26TH DYNASTY, REIGN OF PSAMTIK I,...",8 ¼ in. (21 cm.) high,,"estimate £80,000 - £120,000($106,560 - $159,840)","price realized £387,500 ($521,575)"
6,Lot 15,A CYPRIOT LIMESTONE HEAD OF A MALE VOTARY,"ARCHAIC PERIOD, CIRCA LATE 6TH CENTURY B.C.",7 5/16 in. (18.6 cm.) high,,"estimate £40,000 - £60,000($53,280 - $79,920)","price realized £50,000 ($67,300)"
7,Lot 16,A CYPRIOT POTTERY ANTHROPOMORPHIC JUG,"CYPRO-ARCHAIC I-II, CIRCA 600-480 B.C.",10 ¾ in. (27.3 cm.) high,,"estimate £6,000 - £8,000($7,992 - $10,656)","price realized £21,250 ($28,603)"
8,Lot 17,A CYPRIOT BICHROME POTTERY JUG,"ARCHAIC PERIOD, CIRCA LATE 6TH-EARLY 5TH CENTU...",14 1/8 in. (36.1 cm.) high,,"estimate £6,000 - £8,000($7,992 - $10,656)","price realized £12,500 ($16,825)"
9,Lot 18,A CYPRIOT BICHROME POTTERY JUG,"ARCHAIC PERIOD, CIRCA LATE 6TH-EARLY 5TH CENTU...",15 7/8 in. (40.1 cm.) high,,"estimate £5,000 - £7,000($6,660 - $9,324)","price realized £10,000 ($13,460)"


In [13]:
head = [[
        #auction title/cat: 
        re.sub("[\n\r/]", "", soup.find_all("h2",{"class":"browse-sale-title"})[0].get_text(strip=True)),
        #auction id: 
        re.sub("[\n\r/]", "", soup.find_all("span",{"class":"sale-number"})[0].get_text(strip=True)),
        #date: 
        re.sub("[\n\r/]", "", soup.find_all("span",{"class":"sale-date"})[0].get_text(strip=True)),
        #location: 
        re.sub("[\n\r/]", "", soup.find_all("span",{"class":"sale-date"})[0].get_text(strip=True)),
        #total sale: 
        re.sub("[\n\r/]", "", soup.find_all("span",{"id":"spanTotal"})[0].get_text(strip=True)),
        #price realized: 
        re.sub("[\n\r/]", "", soup.find_all("div",{"id":"lots_for_sale"},"strong")[0].get_text(strip=True))
        ]]
df_head = pd.DataFrame(head*(len(lot_data)), columns = ['category','id','date','location','total_sale','nooflots'])

df_head

,category,id,date,location,total_sale,nooflots
0,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864
1,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864
2,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864
3,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864
4,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864
5,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864
6,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864
7,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864
8,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864
9,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864


In [15]:
result = pd.concat([df_head, lot_data], axis=1, join="inner")
result

,category,id,date,location,total_sale,nooflots,0,1,2,3,4,5,6
0,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864,Lot 4,AN EGYPTIAN WHITE FAIENCE TILE WITH A REKHYT BIRD,"NEW KINGDOM, 19TH-20TH DYNASTY, CIRCA 1292-106...",3 5/8 X 4 in. (9.2 x 10.2 cm.),,"estimate £7,000 - £9,000($9,324 - $11,988)","price realized £8,750 ($11,778)"
1,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864,Lot 5,AN EGYPTIAN POLYCHROME WOOD FIGURE OF AN OFFICIAL,"MIDDLE KINGDOM, EARLY 12TH DYNASTY, CIRCA 1981...",12 3/8 in. (31.5 cm.) high,,"estimate £30,000 - £50,000($39,960 - $66,600)","price realized £35,000 ($47,110)"
2,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864,Lot 6,AN EGYPTIAN PAINTED CARTONNAGE MUMMY MASK,"NEW KINGDOM, 19TH DYNASTY, CIRCA 1292-1185 B.C.",8 ½ in. (21.5 cm.) high,,"estimate £15,000 - £20,000($19,980 - $26,640)","price realized £18,750 ($25,238)"
3,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864,Lot 7,AN EGYPTIAN TURQUOISE FAIENCE FLASK,"NEW KINGDOM, 18TH DYNASTY, CIRCA 1550-1292 B.C.",5 in. (12.8 cm.) high,,"estimate £15,000 - £25,000($19,980 - $33,300)","price realized £4,375 ($5,889)"
4,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864,Lot 11,AN EGYPTIAN BRONZE CAT,"PTOLEMAIC PERIOD, CIRCA 332-30 B.C.",6 5/16 in. (16 cm.) high,,"estimate £15,000 - £25,000($19,980 - $33,300)","price realized £27,500 ($37,015)"
5,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864,Lot 13,AN EGYPTIAN BRONZE OSIRIS-PTAH,"LATE PERIOD, 26TH DYNASTY, REIGN OF PSAMTIK I,...",8 ¼ in. (21 cm.) high,,"estimate £80,000 - £120,000($106,560 - $159,840)","price realized £387,500 ($521,575)"
6,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864,Lot 15,A CYPRIOT LIMESTONE HEAD OF A MALE VOTARY,"ARCHAIC PERIOD, CIRCA LATE 6TH CENTURY B.C.",7 5/16 in. (18.6 cm.) high,,"estimate £40,000 - £60,000($53,280 - $79,920)","price realized £50,000 ($67,300)"
7,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864,Lot 16,A CYPRIOT POTTERY ANTHROPOMORPHIC JUG,"CYPRO-ARCHAIC I-II, CIRCA 600-480 B.C.",10 ¾ in. (27.3 cm.) high,,"estimate £6,000 - £8,000($7,992 - $10,656)","price realized £21,250 ($28,603)"
8,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864,Lot 17,A CYPRIOT BICHROME POTTERY JUG,"ARCHAIC PERIOD, CIRCA LATE 6TH-EARLY 5TH CENTU...",14 1/8 in. (36.1 cm.) high,,"estimate £6,000 - £8,000($7,992 - $10,656)","price realized £12,500 ($16,825)"
9,Antiquities,Auction 18864,"16 December 2020, London","16 December 2020, London","SALE TOTAL: 4,337,125 (GBP)",55 Lotsfor Sale 18864,Lot 18,A CYPRIOT BICHROME POTTERY JUG,"ARCHAIC PERIOD, CIRCA LATE 6TH-EARLY 5TH CENTU...",15 7/8 in. (40.1 cm.) high,,"estimate £5,000 - £7,000($6,660 - $9,324)","price realized £10,000 ($13,460)"


-------

### Join all the code

In [16]:
antiquities = [29064, 29150, 28840, 29063, 28980, 28619, 28775, 28620, 28629, 28618, 28783, 28785, 28552, 28917, 28682, 28125, 28597, 28295, 28580, 28201, 28097, 28363, 28178, 28329, 28200, 27821, 27423, 27522, 27422, 27942, 27521, 26822, 26920, 26821, 26919, 26158, 27002, 26172, 26744, 26157, 26171, 25353, 25123, 25307, 25122, 25050, 24491, 24743, 24670, 24618, 24445, 24628, 24269, 24033, 24150, 24337, 24554, 23986, 24219, 24516, 24326, 24268, 23585, 23820, 17246, 23545, 23710, 23760, 23097, 23096, 23264, 23019, 23214, 23416, 22640, 22639, 22908, 22631, 22857, 22937, 22824, 22181, 22180, 22407, 22490, 22121, 22384]
#antiquities_results = []
emptydf = pd.DataFrame()
#lot_data = [] 

url_antiquities_print=[]
for k in antiquities:
    urll='https://www.christies.com/lotfinder/print_sale.aspx?saleid='+str(k)
    url_antiquities_print.append(urll)

In [22]:
appended_dataframe = pd.DataFrame(columns = ['category','id','date','location','total_sale','nooflots',
                                             'lot','title','descr','measures','empty','estimated','realized'])
                                             
for url in url_antiquities_print[:2]:
    #url = url_antiquities_print #iterate over the urll list 
    response = requests.get(url)
    response.status_code
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # get lot info 
    all_data = [] 
    for i in soup.find_all("tr"):
        all_data.append(i.text)
    all_data = [x.strip().split("\n") for x in all_data]
    #df = pd.DataFrame(all_data[1:])

    clean_data = []
    for i in all_data:
        lent=len(i)
        if lent==7:
            clean_data.append(i)
    
    lot_data = pd.DataFrame.from_records(clean_data, columns = ['lot','title','descr','measures','empty','estimated','realized'])
        
    # get header
    head = [[
        #auction title/cat: 
        re.sub("[\n\r/]", "", soup.find_all("h2",{"class":"browse-sale-title"})[0].get_text(strip=True)),
        #auction id: 
        re.sub("[\n\r/]", "", soup.find_all("span",{"class":"sale-number"})[0].get_text(strip=True)),
        #date: 
        re.sub("[\n\r/]", "", soup.find_all("span",{"class":"sale-date"})[0].get_text(strip=True)),
        #location: 
        re.sub("[\n\r/]", "", soup.find_all("span",{"class":"sale-date"})[0].get_text(strip=True)),
        #total sale: 
        re.sub("[\n\r/]", "", soup.find_all("span",{"id":"spanTotal"})[0].get_text(strip=True)),
        #price realized: 
        re.sub("[\n\r/]", "", soup.find_all("div",{"id":"lots_for_sale"},"strong")[0].get_text(strip=True))
        ]]
    df_head = pd.DataFrame(head*(len(lot_data)), columns = ['category','id','date','location','total_sale','nooflots'])

    # append
    result = pd.concat([df_head, lot_data], axis=1, join="inner")
    
    # antiquities_results.append(result)
    appended_dataframe.append(result)
    
    

The code before works until the result (we get the last url result) but not appending and sotring it

## Solve the appending error

In [32]:
appended_dataframe = pd.DataFrame()
                                             
for url in url_antiquities_print:
    #url = url_antiquities_print #iterate over the urll list 
    response = requests.get(url)
    response.status_code
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # get lot info 
    all_data = [] 
    for i in soup.find_all("tr"):
        all_data.append(i.text)
    all_data = [x.strip().split("\n") for x in all_data]
    #df = pd.DataFrame(all_data[1:])

    clean_data = []
    for i in all_data:
        lent=len(i)
        if lent==7:
            clean_data.append(i)
    
    lot_data = pd.DataFrame.from_records(clean_data, columns = ['lot','title','descr','measures','empty','estimated','realized'])
        
    # get header
    head = [[
        #auction title/cat: 
        re.sub("[\n\r/]", "", soup.find_all("h2",{"class":"browse-sale-title"})[0].get_text(strip=True)),
        #auction id: 
        re.sub("[\n\r/]", "", soup.find_all("span",{"class":"sale-number"})[0].get_text(strip=True)),
        #date: 
        re.sub("[\n\r/]", "", soup.find_all("span",{"class":"sale-date"})[0].get_text(strip=True)),
        #location: 
        re.sub("[\n\r/]", "", soup.find_all("span",{"class":"sale-date"})[0].get_text(strip=True)),
        #total sale: 
        re.sub("[\n\r/]", "", soup.find_all("span",{"id":"spanTotal"})[0].get_text(strip=True)),
        #price realized: 
        re.sub("[\n\r/]", "", soup.find_all("div",{"id":"lots_for_sale"},"strong")[0].get_text(strip=True))
        ]]
    df_head = pd.DataFrame(head*(len(lot_data)), columns = ['category','id','date','location','total_sale','nooflots'])

    # append
    result = pd.concat([df_head, lot_data], axis=1, join="inner")
    
    # antiquities_results.append(result)
    appended_dataframe = appended_dataframe.append(result)

In [33]:
appended_dataframe

,category,id,date,location,total_sale,nooflots,lot,title,descr,measures,empty,estimated,realized
0,Antiquities,Auction 19876,"8 December 2021, London","8 December 2021, London","SALE TOTAL: 6,407,750 (GBP)",152 Lotsfor Sale 19876,Lot 1,AN ATTIC BLACK-FIGURED TREFOIL-LIPPED OINOCHOE,"ATTRIBUTED TO THE GELA PAINTER, CIRCA 510-490 ...",6 3⁄4 in. (17.2 cm.) high,,"estimate £6,000 - £8,000($7,956 - $10,608)","price realized £8,125 ($10,749)"
1,Antiquities,Auction 19876,"8 December 2021, London","8 December 2021, London","SALE TOTAL: 6,407,750 (GBP)",152 Lotsfor Sale 19876,Lot 2,A GREEK MEGARIAN WARE POTTERY BOWL,"HELLENISTIC PERIOD, CIRCA 2ND CENTURY B.C.",5 5⁄8 in. (14.2 cm.) diameter,,estimate £400 - £600($530 - $795),"price realized £1,500 ($1,985)"
2,Antiquities,Auction 19876,"8 December 2021, London","8 December 2021, London","SALE TOTAL: 6,407,750 (GBP)",152 Lotsfor Sale 19876,Lot 3,AN ATTIC WHITE-GROUND LEKYTHOS,CIRCA MID 5TH CENTURY B.C.,11 1⁄2 in. (29.2 cm.) high,,"estimate £8,000 - £12,000($10,608 - $15,912)","price realized £18,750 ($24,806)"
3,Antiquities,Auction 19876,"8 December 2021, London","8 December 2021, London","SALE TOTAL: 6,407,750 (GBP)",152 Lotsfor Sale 19876,Lot 5,SEVEN GREEK TERRACOTTAS,"HELLENISTIC PERIOD, CIRCA 3RD-2ND CENTURY B.C.",4 3⁄8 in. (11 cm.) long max.,,"estimate £2,500 - £3,500($3,315 - $4,641)","price realized £4,375 ($5,788)"
4,Antiquities,Auction 19876,"8 December 2021, London","8 December 2021, London","SALE TOTAL: 6,407,750 (GBP)",152 Lotsfor Sale 19876,Lot 6,AN ETRUSCAN BRONZE HANDLE FRAGMENT,CIRCA 5TH-4TH CENTURY B.C.,8 3⁄8 in. (21.3 cm.) high,,"estimate £1,500 - £2,500($1,989 - $3,315)","price realized £2,500 ($3,308)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,Antiquities Including the Plesch Collection of...,Auction 5951,"28 April 2009, London, Sout...","28 April 2009, London, Sout...","SALE TOTAL: 1,036,687 (GBP)",200 Lotsfor Sale 5951,Lot 243,A GREEK STYLE POTTERY MULE HEAD RHYTON,AFTER THE ANTIQUE,8¼ in. (21 cm.) and 12 in. (30.5 cm.) high res...,,"estimate £600 - £900($872 - $1,309)","price realized £2,000 ($2,902)"
167,Antiquities Including the Plesch Collection of...,Auction 5951,"28 April 2009, London, Sout...","28 April 2009, London, Sout...","SALE TOTAL: 1,036,687 (GBP)",200 Lotsfor Sale 5951,Lot 244,A LARGE ATTIC BLACK GLAZED POTTERY HYDRIA,4TH CENTURY B.C.,24 in. (61 cm.) and 7½ in. (19 cm.) high respe...,,"estimate £3,000 - £5,000($4,362 - $7,270)","price realized £6,875 ($9,976)"
168,Antiquities Including the Plesch Collection of...,Auction 5951,"28 April 2009, London, Sout...","28 April 2009, London, Sout...","SALE TOTAL: 1,036,687 (GBP)",200 Lotsfor Sale 5951,Lot 245,"AN APULIAN RED-FIGURE FISH PLATE, ATTRIBUTED T...",CIRCA 330 B.C.,9 in. (23 cm.) diam.,,"estimate £1,500 - £2,500($2,181 - $3,635)","price realized £3,250 ($4,716)"
169,Antiquities Including the Plesch Collection of...,Auction 5951,"28 April 2009, London, Sout...","28 April 2009, London, Sout...","SALE TOTAL: 1,036,687 (GBP)",200 Lotsfor Sale 5951,Lot 246,"AN APULIAN RED-FIGURE KANTHAROS, ATTRIBUTED TO...",CIRCA 4TH CENTURY B.C.,13 in. (33 cm.) high,,"estimate £2,000 - £2,500($2,908 - $3,635)","price realized £4,750 ($6,892)"


In [34]:
len(appended_dataframe)

10379

In [35]:
appended_dataframe.reset_index(drop=True)

,category,id,date,location,total_sale,nooflots,lot,title,descr,measures,empty,estimated,realized
0,Antiquities,Auction 19876,"8 December 2021, London","8 December 2021, London","SALE TOTAL: 6,407,750 (GBP)",152 Lotsfor Sale 19876,Lot 1,AN ATTIC BLACK-FIGURED TREFOIL-LIPPED OINOCHOE,"ATTRIBUTED TO THE GELA PAINTER, CIRCA 510-490 ...",6 3⁄4 in. (17.2 cm.) high,,"estimate £6,000 - £8,000($7,956 - $10,608)","price realized £8,125 ($10,749)"
1,Antiquities,Auction 19876,"8 December 2021, London","8 December 2021, London","SALE TOTAL: 6,407,750 (GBP)",152 Lotsfor Sale 19876,Lot 2,A GREEK MEGARIAN WARE POTTERY BOWL,"HELLENISTIC PERIOD, CIRCA 2ND CENTURY B.C.",5 5⁄8 in. (14.2 cm.) diameter,,estimate £400 - £600($530 - $795),"price realized £1,500 ($1,985)"
2,Antiquities,Auction 19876,"8 December 2021, London","8 December 2021, London","SALE TOTAL: 6,407,750 (GBP)",152 Lotsfor Sale 19876,Lot 3,AN ATTIC WHITE-GROUND LEKYTHOS,CIRCA MID 5TH CENTURY B.C.,11 1⁄2 in. (29.2 cm.) high,,"estimate £8,000 - £12,000($10,608 - $15,912)","price realized £18,750 ($24,806)"
3,Antiquities,Auction 19876,"8 December 2021, London","8 December 2021, London","SALE TOTAL: 6,407,750 (GBP)",152 Lotsfor Sale 19876,Lot 5,SEVEN GREEK TERRACOTTAS,"HELLENISTIC PERIOD, CIRCA 3RD-2ND CENTURY B.C.",4 3⁄8 in. (11 cm.) long max.,,"estimate £2,500 - £3,500($3,315 - $4,641)","price realized £4,375 ($5,788)"
4,Antiquities,Auction 19876,"8 December 2021, London","8 December 2021, London","SALE TOTAL: 6,407,750 (GBP)",152 Lotsfor Sale 19876,Lot 6,AN ETRUSCAN BRONZE HANDLE FRAGMENT,CIRCA 5TH-4TH CENTURY B.C.,8 3⁄8 in. (21.3 cm.) high,,"estimate £1,500 - £2,500($1,989 - $3,315)","price realized £2,500 ($3,308)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10374,Antiquities Including the Plesch Collection of...,Auction 5951,"28 April 2009, London, Sout...","28 April 2009, London, Sout...","SALE TOTAL: 1,036,687 (GBP)",200 Lotsfor Sale 5951,Lot 243,A GREEK STYLE POTTERY MULE HEAD RHYTON,AFTER THE ANTIQUE,8¼ in. (21 cm.) and 12 in. (30.5 cm.) high res...,,"estimate £600 - £900($872 - $1,309)","price realized £2,000 ($2,902)"
10375,Antiquities Including the Plesch Collection of...,Auction 5951,"28 April 2009, London, Sout...","28 April 2009, London, Sout...","SALE TOTAL: 1,036,687 (GBP)",200 Lotsfor Sale 5951,Lot 244,A LARGE ATTIC BLACK GLAZED POTTERY HYDRIA,4TH CENTURY B.C.,24 in. (61 cm.) and 7½ in. (19 cm.) high respe...,,"estimate £3,000 - £5,000($4,362 - $7,270)","price realized £6,875 ($9,976)"
10376,Antiquities Including the Plesch Collection of...,Auction 5951,"28 April 2009, London, Sout...","28 April 2009, London, Sout...","SALE TOTAL: 1,036,687 (GBP)",200 Lotsfor Sale 5951,Lot 245,"AN APULIAN RED-FIGURE FISH PLATE, ATTRIBUTED T...",CIRCA 330 B.C.,9 in. (23 cm.) diam.,,"estimate £1,500 - £2,500($2,181 - $3,635)","price realized £3,250 ($4,716)"
10377,Antiquities Including the Plesch Collection of...,Auction 5951,"28 April 2009, London, Sout...","28 April 2009, London, Sout...","SALE TOTAL: 1,036,687 (GBP)",200 Lotsfor Sale 5951,Lot 246,"AN APULIAN RED-FIGURE KANTHAROS, ATTRIBUTED TO...",CIRCA 4TH CENTURY B.C.,13 in. (33 cm.) high,,"estimate £2,000 - £2,500($2,908 - $3,635)","price realized £4,750 ($6,892)"


In [36]:
appended_dataframe.to_csv(r'appended_dataframe.csv', index = False, header = True)